In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/Users/piotr/Documents/studia/mgr/CVLab/cl-teacher-adaptation


/Users/piotr/Documents/studia/mgr/CVLab/cl-teacher-adaptation/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from pathlib import Path

p = Path("models/cifar100_5/resnet34/")
expname_path = {}
for dir in p.iterdir():
    for i in dir.iterdir():
        path = i.readlink()
        exp_name = "_".join(i.parts[2:])
        expname_path[exp_name] = path

In [3]:
resnet_finetuning = {
    "resnet34_finetuning_reg": "results/2024/04.24/13-35-10/0/cifar100_fixed_finetuning",
    "resnet34_finetuning_noreg": "results/2024/04.17/18-13-25/0/cifar100_fixed_finetuning_final_checkpoint",
}

In [5]:
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import torch
from tqdm import tqdm

from src.analysis import core
from src.analysis.svd import torch_calculate_eigens_directclr
import re

get_digits = lambda str: int(re.sub(r"\D", "", str))
DEVICE = "cpu"
NUM_CLASSES = 20


def get_activs(exp_name, path):
    root = Path("activations")
    save_path = root.joinpath(*exp_name.split("_"))
    save_path.mkdir(exist_ok=True, parents=True)

    train_tasks_data = defaultdict(dict)
    test_tasks_data = defaultdict(dict)
    cfg = core.create_cfg(path)
    cfg.data.num_workers = 8
    data_factory = core.DataFactory(cfg)
    model_factory = core.ModelFactory(cfg, path, device=DEVICE)
    for task_idx, ckpt in tqdm(enumerate(sorted(model_factory.ckpts))):
        model = model_factory.create_model(task=task_idx, num_classes=NUM_CLASSES)
        assert task_idx == get_digits(ckpt.name)
        train_loaders, _, test_loaders, _ = data_factory[: task_idx + 1]
        for data_idx, (train_loader, test_loader) in enumerate(
            zip(train_loaders, test_loaders)
        ):
            train_outs = core.get_activations(model.model, train_loader, device=DEVICE)
            test_outs = core.get_activations(model.model, test_loader, device=DEVICE)

            train_tasks_data[task_idx][data_idx] = train_outs
            test_tasks_data[task_idx][data_idx] = test_outs
    torch.save(train_tasks_data, save_path / "train.pth")
    torch.save(test_tasks_data, save_path / "test.pth")
    return train_tasks_data, test_tasks_data

In [6]:
root = Path("activations")
save_path = root.joinpath(*exp_name.split("_"))

In [7]:
for e_name, path in resnet_finetuning.items():
    tasks_data = get_activs(exp_name=e_name, path=path)
    break

Files already downloaded and verified
Files already downloaded and verified


0it [00:00, ?it/s]

<All keys matched successfully>


100%|██████████| 16/16 [01:27<00:00,  5.44s/it]
0it [01:53, ?it/s]
